## Zadatak 1(15)
U danom rječniku SK_rjecnik.txt (koji se nalazi u mapi /usr/share/RacJez/ispitna turing.mathos.hr) uz pomoć regularnih izraza, izvuči samo imenice sa oblikom riječi (riječs naglascima), vrsta i rod, gramatička obilježja i definicijom te ih spremiti u json datoteku.

Primjer:
```
gaj.json{
    "rijec":"gȃj",
    "vrsta":"imenica",
    "rod":"m",
    "gramatička obiljezja":"G gája mn. N gájevi G gájēvā",
    "definicija":"mlada šuma; sin. lug¹, šumarak"}
```

In [6]:
import re
import json

file_path = 'C:/Users/Danijela Valjak/Desktop/3.godina/1.semestar/jezikoslovlje/SK_rjecnik.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

# Regularni izraz za pronalaženje imeničkih unosa
pattern = re.compile(r'(?P<rijec>[\w\s]+?)\s+im\.\s(?P<rod>\w\.)\s+(?P<gramatika>.+?)\s+(?P<definicija>\[.+?\].+?)(?=\ng|\Z)', re.DOTALL | re.MULTILINE)

matches = pattern.finditer(content)

nouns_data = []
for match in matches:
    rijec = match.group('rijec').strip()
    vrsta = 'imenica'
    rod = match.group('rod').strip()
    gramaticka_obiljezja = match.group('gramatika').strip()
    definicija = match.group('definicija').strip()
    
    noun_info = {
        "rijec": rijec,
        "vrsta": vrsta,
        "rod": rod,
        "gramatička obilježja": gramaticka_obiljezja,
        "definicija": definicija
    }
    nouns_data.append(noun_info)

# Ispisivanje u JSON datoteku
output_file = 'gaj.json'
with open(output_file, 'w', encoding='utf-8') as outfile:
    json.dump(nouns_data, outfile, ensure_ascii=False, indent=4)


## Zadatak 2 (15)
Napišite gramatiku obogaćenu značajkama koja opisuje sljedeće rečenice:
```
Znatiželjan pas je lajao
Umiljata mačka glasno prede
Umiljata mačka je glasno prela
```
Vodite računa o sročnosti NP sa VP u broju, rodu i licu. Dakle, gramatika ne bi trebala prihvatiti rečenicu Znatiželjna pas je glasno  lajao ili Znatiželjan pas jeglasno lajala ne treba prihvatiti.

In [15]:
import nltk
from nltk2tikz import show_parse
import random

ModuleNotFoundError: No module named 'nltk2tikz'

In [21]:
%%writefile grammar.fcfg
# grammar.fcfg
% start S

S -> NP[broj=sg, rod=?r] VP[broj=sg, rod=?r]

NP[broj=sg, rod=m] -> Adj[broj=sg, rod=m] N[broj=sg, rod=m]
NP[broj=sg, rod=z] -> Adj[broj=sg, rod=z] N[broj=sg, rod=z]

VP[broj=sg, rod=m] -> Aux V[broj=sg, rod=m] | V[broj=sg, rod=m]
VP[broj=sg, rod=z] -> Aux V[broj=sg, rod=z] | V[broj=sg, rod=z]

Adj[broj=sg, rod=m] -> 'Znatiželjan'
Adj[broj=sg, rod=z] -> 'Umiljata'

N[broj=sg, rod=m] -> 'pas'
N[broj=sg, rod=z] -> 'mačka'

Aux -> 'je'

V[broj=sg, rod=m] -> 'lajao'
V[broj=sg, rod=z] -> 'prela' | 'glasno prede' | 'je glasno prela'



Overwriting grammar.fcfg


In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Učitavanje parsera
cp = nltk.load_parser('grammar.fcfg', trace=0)
print(cp.grammar())

# Parsiranje rečenice
sentence = 'Znatiželjan pas je lajao'
tokens = word_tokenize(sentence)

# Parsiranje rečenice
trees = list(cp.parse(tokens))

# Prikazivanje stabla
if trees:
    tree = trees[0]
    tree.pretty_print()
else:
    print("No parse tree found.")


## Zadatak 3 (15)

Implementirajte analizu sentimenta koristeći Bayesov klasifikator za popis filmskih recenzija koje se nalaze u nltk.corpus.movie_reviews. Za značajke pojedine recenzije koristite informaciju sadrži li recenzija najčešćih 2000 riječi iz movie_reviews korpusa. Prijavite rezultate na sljedeći način:Točnost (accuracy) klasifikatora mora biti barem 70%. Prikažite preciznost, odziv i ocjenu za pojedinu kategoriju pozitivnog i negativnog sentimenta.F1
Prikažite matricu zbunjenosti

In [23]:
import nltk
from nltk.corpus import movie_reviews
import random
from nltk import FreqDist

# Učitavanje korpusa movie_reviews
nltk.download('movie_reviews')


[nltk_data] Downloading package movie_reviews to C:\Users\Danijela
[nltk_data]     Valjak\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [24]:
# Izgradnja skupa podataka
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

# Slučajno promiješajmo dokumente kako bismo izbjegli bilo kakav redoslijed u podacima
random.shuffle(documents)


In [25]:
# Izgradnja popisa najčešćih 2000 riječi
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words.keys())[:2000]

# Funkcija za ekstrakciju značajki
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features


In [26]:
# Podijela podataka na skup za obuku i skup za testiranje (80-20)
train_set = [(document_features(d), c) for (d,c) in documents[:1600]]
test_set = [(document_features(d), c) for (d,c) in documents[1600:]]
# Treniranje Bayesovog klasifikatora
classifier = nltk.NaiveBayesClassifier.train(train_set)

# Evaluacija klasifikatora
print("Točnost (accuracy):", nltk.classify.accuracy(classifier, test_set))

# Prikaz preciznosti, odziva i F1 ocjene za svaku klasu
refsets = nltk.collections.defaultdict(set)
testsets = nltk.collections.defaultdict(set)

for i, (feats, label) in enumerate(test_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)

print("Preciznost za negativan sentiment:", nltk.precision(refsets['neg'], testsets['neg']))
print("Preciznost za pozitivan sentiment:", nltk.precision(refsets['pos'], testsets['pos']))

print("Odziv za negativan sentiment:", nltk.recall(refsets['neg'], testsets['neg']))
print("Odziv za pozitivan sentiment:", nltk.recall(refsets['pos'], testsets['pos']))

print("F1 ocjena za negativan sentiment:", nltk.f_measure(refsets['neg'], testsets['neg']))
print("F1 ocjena za pozitivan sentiment:", nltk.f_measure(refsets['pos'], testsets['pos']))


Točnost (accuracy): 0.7875
Preciznost za negativan sentiment: 0.8055555555555556
Preciznost za pozitivan sentiment: 0.7663043478260869
Odziv za negativan sentiment: 0.8018433179723502
Odziv za pozitivan sentiment: 0.7704918032786885
F1 ocjena za negativan sentiment: 0.8036951501154733
F1 ocjena za pozitivan sentiment: 0.768392370572207


## Zadatak 4(15)
Neka je dan sljedeći skup rečenica (korpus).

    Google and Facebook are strangling the free press to death.Democracy is the loserYour 60-second guide to security stuff Google touted today at Next'18A Guide to Using Android Without Selling Your Soul to GoogleReview: Lenovo’s Google Smart Display is pretty and intelligentGoogle Maps user spots mysterious object submerged off the coastof Greece - and no-one knows what it isAndroid is better than IOSIn information retrieval, tf–idf or TFIDF, short for termfrequency–inverse document frequencyis a numerical statistic that is intended to reflecthow important a word is to a document in a collection or corpus.It is often used as a weighting factor in searches of informationretrievaltext mining, and user modeling. The tf-idf value increasesproportionallyto the number of times a word appears in the documentand is offset by the frequency of the word in the corpus

Učinte sljedeće: 1) Reprezentirajte svaku rečenicu TF-IDF mjerom, te napravite klasteriranje u2 klastera. Ispišite u kojem se klasteru nalazi svaka rečenica iz korpusa. 2) Odredite kojimaklasterima pripadaju sljedeće rečenice:tf-idf is used in used in information retrieval. iOS is betterthan Android.Napomena: vodite računa o normalizaciji tokena i izbacivanju učestalih tokena prijeizračuna TF-IDF mjere.

In [27]:
%pip install scikit-learn



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Korpus rečenica
sentences = [
    "Google and Facebook are strangling the free press to death.",
    "Democracy is the loser",
    "Your 60-second guide to security stuff Google touted today at Next '18",
    "A Guide to Using Android Without Selling Your Soul to Google",
    "Review: Lenovo’s Google Smart Display is pretty and intelligent",
    "Google Maps user spots mysterious object submerged off the coast of Greece - and no-one knows what it is",
    "Android is better than IOS",
    "In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. It is often used as a weighting factor in searches of information retrieval text mining, and user modeling. The tf-idf value increases proportionally to the number of times a word appears in the document and is offset by the frequency of the word in the corpus"
]

# 1. Izračun TF-IDF mjere za svaku rečenicu
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(sentences)

# 2. Klasteriranje rečenica u 2 klastera
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(X)

# 3. Ispis u kojem se klasteru nalazi svaka rečenica iz korpusa
for i, sentence in enumerate(sentences):
    cluster = kmeans.labels_[i]
    print(f"Rečenica '{sentence}' je u klasteru {cluster}")

# 4. Određivanje kojima klasterima pripadaju dodatne rečenice
new_sentences = [
    "tf-idf is used in used in information retrieval.",
    "iOS is better than Android."
]

# Izračunavanje TF-IDF mjere za nove rečenice
new_X = vectorizer.transform(new_sentences)

# Klasteriranje novih rečenica
predicted_clusters = kmeans.predict(new_X)

# Ispis rezultata
for i, sentence in enumerate(new_sentences):
    cluster = predicted_clusters[i]
    print(f"Rečenica '{sentence}' pripada klasteru {cluster}")


Rečenica 'Google and Facebook are strangling the free press to death.' je u klasteru 0
Rečenica 'Democracy is the loser' je u klasteru 0
Rečenica 'Your 60-second guide to security stuff Google touted today at Next '18' je u klasteru 0
Rečenica 'A Guide to Using Android Without Selling Your Soul to Google' je u klasteru 1
Rečenica 'Review: Lenovo’s Google Smart Display is pretty and intelligent' je u klasteru 0
Rečenica 'Google Maps user spots mysterious object submerged off the coast of Greece - and no-one knows what it is' je u klasteru 0
Rečenica 'Android is better than IOS' je u klasteru 1
Rečenica 'In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. It is often used as a weighting factor in searches of information retrieval text mining, and user modeling. The tf-idf value increases proportionally to the number of times a wo